<a href="https://colab.research.google.com/github/indrad123/imagecaptioning/blob/main/fin_translation_marianmt_flickr30k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flickr30k Datasets

In [1]:
# calculate number of cores
import multiprocessing

num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 8


In [2]:
# Step 1: Install required libraries
!pip install datasets transformers torch sacremoses


In [1]:
# Step 1: Install required libraries
#!pip install datasets transformers torch pandas h5py

import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import os

# Step 2: Load the original dataset
dataset = load_dataset("Mozilla/flickr30k-transformed-captions")

# Step 3: Check dataset splits and load the 'test' split if it exists
if 'test' in dataset:
    dataset_split = dataset['test']
else:
    raise KeyError("The dataset does not contain a 'test' split.")

# Convert dataset_split to a list of dictionaries
dataset_list = dataset_split.to_pandas().to_dict(orient='records')

# Step 4: Load the pre-trained translation model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-id'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Step 5: Move the model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Function to translate text using the pre-trained model
def translate_text(texts, tokenizer, model, device):
    if not texts:
        return []
    # Ensure all texts are strings
    texts = [str(text) for text in texts]
    batch = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    translated = model.generate(**batch)
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_texts

# Helper function to process a chunk of the dataset
def process_chunk(chunk):
    alt_texts = []
    original_texts = []
    for record in chunk:
        if 'alt_text' in record and 'original_alt_text' in record:  # Ensure correct key usage
            alt_texts.append(record['alt_text'])
            original_texts.append(record['original_alt_text'])

    if not alt_texts or not original_texts:
        return chunk  # Return the chunk as-is if there are no texts to process

    alt_texts_translated = translate_text(alt_texts, tokenizer, model, device)
    original_texts_translated = translate_text(original_texts, tokenizer, model, device)

    for i, record in enumerate(chunk):
        if 'alt_text' in record and 'original_alt_text' in record:  # Ensure correct key usage
            record['alt_text_id'] = alt_texts_translated[i]
            record['original_alt_text_id'] = original_texts_translated[i]

    return chunk

# Function to split the dataset into smaller chunks for sequential processing
def split_dataset(dataset, chunk_size):
    return [dataset[i:i + chunk_size] for i in range(0, len(dataset), chunk_size)]

# Directory to save intermediate results
os.makedirs("intermediate_results", exist_ok=True)

# Determine the chunk size
chunk_size = 50  # Adjust this size based on available memory and disk space

# Split the dataset into smaller chunks
dataset_chunks = split_dataset(dataset_list, chunk_size)

# Process each chunk sequentially and save intermediate results
for i, chunk in enumerate(dataset_chunks):
    translated_chunk = process_chunk(chunk)
    df_chunk = pd.DataFrame(translated_chunk)
    df_chunk.to_hdf(f"intermediate_results/chunk_{i}.h5", key='df', mode='w')

# Combine all intermediate results
translated_data = []
for i in range(len(dataset_chunks)):
    df_chunk = pd.read_hdf(f"intermediate_results/chunk_{i}.h5", key='df')
    translated_data.extend(df_chunk.to_dict(orient='records'))

# Convert the translated data to a Dataset
translated_dataset = Dataset.from_pandas(pd.DataFrame(translated_data))

# Create a DatasetDict
translated_dataset_dict = DatasetDict({
    'test': translated_dataset
})

# Push the dataset to the Hugging Face Hub -->>> Used to Push to Hugging Face
# translated_dataset_dict.push_to_hub("indrad123/flickr30k-transformed-captions-indonesia")


<ipython-input-1-3594bb4caadf>:82: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['image', 'alt_text', 'sentids', 'split', 'img_id', 'filename',
       'original_alt_text', 'alt_text_id', 'original_alt_text_id'],
      dtype='object')]

  df_chunk.to_hdf(f"intermediate_results/chunk_{i}.h5", key='df', mode='w')


Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

HfHubHTTPError:  (Request ID: Root=1-66647558-25d9ea8b46dec3717d1ff298;1f7d03d8-c9f8-49b7-ad0b-ca13ac7c90b5)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/datasets/indrad123/flickr30k-transformed-captions-indonesia/preupload/main.
If you are trying to create or update content,make sure you have a token with the `write` role.

# Save to Google Drive

In [11]:
# Save the translated dataset to a local directory
output_dir = "/content/drive/MyDrive/Datasets/flickr30k_marianmt_translation"
translated_dataset_dict.save_to_disk(output_dir)

print(f"Translated dataset saved to {output_dir}")

Saving the dataset (0/9 shards):   0%|          | 0/31014 [00:00<?, ? examples/s]

Translated dataset saved to /content/drive/MyDrive/Datasets/flickr30k_marianmt_translation


# Test to Reload from Google Drive

In [13]:
from datasets import load_from_disk

disk_translated_dataset_dict = load_from_disk(output_dir)


In [16]:
disk_translated_dataset_dict['test'][0]

{'image': {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x03\x02\x02\x02\x02\x02\x04\x03\x03\x02\x03\x05\x04\x05\x05\x05\x04\x04\x04\x05\x06\x07\x06\x05\x05\x07\x06\x04\x04\x06\t\x06\x07\x08\x08\x08\x08\x08\x05\x06\t\n\t\x08\n\x07\x08\x08\x08\xff\xdb\x00C\x01\x01\x01\x01\x02\x02\x02\x04\x02\x02\x04\x08\x05\x04\x05\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\xff\xc0\x00\x11\x08\x01\xf4\x01M\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x02\x03\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x07\x08\x05\x06\t\x03\x04\n\x02\x01\x00\x0b\xff\xc4\x00D\x10\x00\x02\x02\x02\x02\x01\x03\x04\x00\x04\x04\x04\x04\x04\x02\x0b\x02\x03\x01\x04\x05\x06\x07\x11\x12\x08\x13!\x00\x14"1\t\x15#A\x162Qa$Bq\x81\x173R\x91\n%Cb\xa14\

# Test to Load from Hugging Face

In [24]:
test_indrad123 = load_dataset("indrad123/flickr30k-transformed-captions-indonesia")

Generating test split: 0 examples [00:00, ? examples/s]

In [25]:
test_indrad123['test'][0]

{'image': {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x03\x02\x02\x02\x02\x02\x04\x03\x03\x02\x03\x05\x04\x05\x05\x05\x04\x04\x04\x05\x06\x07\x06\x05\x05\x07\x06\x04\x04\x06\t\x06\x07\x08\x08\x08\x08\x08\x05\x06\t\n\t\x08\n\x07\x08\x08\x08\xff\xdb\x00C\x01\x01\x01\x01\x02\x02\x02\x04\x02\x02\x04\x08\x05\x04\x05\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\xff\xc0\x00\x11\x08\x01\xf4\x01M\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x02\x03\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x07\x08\x05\x06\t\x03\x04\n\x02\x01\x00\x0b\xff\xc4\x00D\x10\x00\x02\x02\x02\x02\x01\x03\x04\x00\x04\x04\x04\x04\x04\x02\x0b\x02\x03\x01\x04\x05\x06\x07\x11\x12\x08\x13!\x00\x14"1\t\x15#A\x162Qa$Bq\x81\x173R\x91\n%Cb\xa14\